In [1]:
import numpy as np
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
import tensorflow as tf

In [2]:
import os
print (os.getcwd())
print(os.getcwd())

C:\Users\ailab\1projects\MusicGan\styleMuseGAN
C:\Users\ailab\1projects\MusicGan\styleMuseGAN


In [3]:
all_track=[]

In [4]:
i=0
k=0
c=20000
batch_size=500

In [5]:
# analizing notes
#input a Multitrack object 
#ex:
#another_multitrack = Multitrack('./data2/rock-39.mid')
#note_distribution(another_multitrack)
#return note_distribution of the whole song which order is C #C D #D ... bB B

def note_distribution(another_multitrack):
    #notes_per_beat is the minus 24 mod14 product C(Do) to be zero is better for the further task
    notes_per_beat=[]
    length=len(another_multitrack.get_merged_pianoroll())
    merged=another_multitrack.get_merged_pianoroll()
    #print(merged)

    for i in range(length):
        #if (length-i)%1000==0:
            #print(length-i)
        #print('ori',np.nonzero(another_multitrack.get_merged_pianoroll()[i])[0])
        temp=list(np.nonzero(merged[i])[0])              
        for j in range(len(temp)):
            temp[j]=(temp[j]-24)%12
        notes_per_beat.append(temp)
        #print(temp)
        #print('new',temp)
        #print(list(np.nonzero(another_multitrack.get_merged_pianoroll()[i])[0])+1)
    #np.nonzero(another_multitrack.get_merged_pianoroll())
    length=len(another_multitrack.get_merged_pianoroll())
    note_distributions=[]
    for i in range(12):
        note_distributions.append(0)
    #print(note_distributions)
    for i in range(length):
        #print(notes_per_beat[i])
        for j in range(12):
            #print(j,notes_per_beat[i].count(j))
            note_distributions[j]= notes_per_beat[i].count(j)+note_distributions[j]
    #print(note_distributions)
    sum=0
    for ele in note_distributions:
        sum=sum+ele
    note_distribution_frequency=[]
    for ele in note_distributions:
        note_distribution_frequency.append(ele/sum)
    
    return note_distributions,note_distribution_frequency


In [ ]:

file=open('file_path2.txt','rb')
i=0
note_dis=[]
length=0
for line in file:
    i=i+1    
    #c=c-1
    #if c%100==0:
        #print(c/100)
    #print(c)
    #print(line[:-2].decode('utf-8'))
    loaded = Multitrack(line[:-2].decode('utf-8'))
    #print(loaded)
    #file_names.append(line[:-2].decode('utf-8'))
    #note_dis.append(note_distribution(loaded))
    temp=loaded.get_merged_pianoroll()
    #####
    #Computing longest length
    ###
    #print(temp.shape)
    if length<temp.shape[0]:
        length=temp.shape[0]
    if i%20==0:
        print(i)
print(length)



20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
422

In [14]:
print(len(note_dis))
print(note_dis[0])

21425
([7215, 8, 7598, 0, 3028, 7203, 1241, 8557, 0, 8972, 0, 3465], [0.15257893289910546, 0.0001691796899782181, 0.16067841055681265, 0.0, 0.06403451265675555, 0.15232516336413812, 0.026243999407871085, 0.18095882589295156, 0.0, 0.18973502231057163, 0.0, 0.07327595322181572])


In [16]:
#save data
"""
import pickle
file = open('note_dis_dataset.pke', 'wb')
pickle.dump(note_dis, file)
file.close()
"""


In [17]:
#load data
with open('note_dis_dataset.pke', 'rb') as file:
    note_dis =pickle.load(file)

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((note_dis))
dataset=dataset.shuffle(50).batch(batch_size).repeat(10)
iterator =dataset.make_one_shot_iterator()
next_element=iterator.get_next()

In [36]:
with tf.Session() as sess:
    for i in range(10):
        print(sess.run(next_element)[1])

[[8.9310000e+03 4.9050000e+03 2.2153000e+04 9.6400000e+02 7.5690000e+03
  1.2387000e+04 8.4500000e+02 1.3954000e+04 1.1540000e+03 1.7811000e+04
  7.5520000e+03 7.0600000e+02]
 [9.0275042e-02 4.9580012e-02 2.2392374e-01 9.7441655e-03 7.6507866e-02
  1.2520848e-01 8.5413065e-03 1.4104781e-01 1.1664696e-02 1.8003456e-01
  7.6336034e-02 7.1362867e-03]]
[[6.06400000e+03 5.75100000e+03 6.40600000e+03 4.59200000e+03
  1.37750000e+04 3.30800000e+03 1.02750000e+04 5.28000000e+03
  7.78200000e+03 9.67500000e+03 5.21000000e+02 1.17930000e+04]
 [7.11553320e-02 6.74825758e-02 7.51683861e-02 5.38828000e-02
  1.61636665e-01 3.88162695e-02 1.20567456e-01 6.19558319e-02
  9.13144499e-02 1.13527022e-01 6.11344492e-03 1.38379768e-01]]
[[1.6230000e+03 0.0000000e+00 4.4300000e+02 1.1410000e+03 0.0000000e+00
  5.7800000e+02 0.0000000e+00 1.5670000e+03 1.3590000e+03 0.0000000e+00
  2.6100000e+02 1.6000000e+02]
 [2.2756590e-01 0.0000000e+00 6.2114414e-02 1.5998317e-01 0.0000000e+00
  8.1043184e-02 0.0000000e+